In [1]:
epochs = 1
n_train_items = 6000
rounds = 650
total_client = 100
C = 0.1
n_workers = int(total_client * C)
batch_size = 600

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
from torch.utils.tensorboard import SummaryWriter
import random

/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

In [3]:
import syft as sy  # <-- NEW: import the Pysyft library
hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning
# simulation functions
def connect_to_workers(n_workers):
    return [
        sy.VirtualWorker(hook, id=f"worker{i+1}")
        for i in range(n_workers)
    ]


workers = connect_to_workers(n_workers=n_workers)


/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [4]:
class Arguments():
    def __init__(self):
        self.batch_size = batch_size
        self.test_batch_size = 60
        self.epochs = epochs
        self.rounds = rounds
        self.lr = 0.02
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 2
        self.save_model = False
        self.n_train_items = n_train_items

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [5]:
# federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
#     datasets.MNIST('../data', train=True, download=True,
#                    transform=transforms.Compose([
#                        transforms.ToTensor(),
#                        transforms.Normalize((0.1307,), (0.3081,))
#                    ]))
#     .federate(workers), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
#     batch_size=args.batch_size, shuffle=True, **kwargs)

# test_loader = torch.utils.data.DataLoader(
#     datasets.MNIST('../data', train=False, transform=transforms.Compose([
#                        transforms.ToTensor(),
#                        transforms.Normalize((0.1307,), (0.3081,))
#                    ])),
#     batch_size=args.test_batch_size, shuffle=True, **kwargs)

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../../data', train=True, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size
)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../../data', train=False, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size
)

    
#---

# less_train_dataloader = [
#         ((data), (target))
#         for i, (data, target) in enumerate(train_loader)
#         if i < (n_train_items / args.batch_size) * 10
#     ]





In [6]:
print(len(train_loader))
# print(len(less_train_dataloader))

100


In [7]:
# from PIL import Image
# import numpy 
# #mnist_dataset.__getitem__(2)[1]
# a = (mnist_dataset.__getitem__(0)[0]).numpy()
# a.dtype = 'uint8'
# print(a)
# Image.fromarray(a[0], mode= 'P')

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5, 1)
        self.conv2 = nn.Conv2d(32, 64, 5, 1)
        self.fc1 = nn.Linear(4*4*64, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*64)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [9]:
def model_init(workers, Net):
    model_list = list()
    for worker in workers:
        model_list.append(Net)
    return model_list
def opt_init(model_list):
    opt_list = list()
    for model  in model_list:
        opt_list.append(optim.SGD(model.parameters(), lr=args.lr))
    return opt_list
def random_sample(train_dataloader):
    choice_list = sorted(random.sample(range(100), 10))
    count = 0
    tmp = []
    for i, data in enumerate(train_dataloader):
        if  i == choice_list[count]:
            tmp.append(data)
            if count == 9:
                pass
            else:
                count += 1
    return tmp


In [11]:
def train(args, device, train_loader, opt_list, workers):
    global model_list
    ## start training and record the model into model_list
    
    less_train_dataloader = random_sample(train_loader)
    for epoch in range(args.epochs):
        for batch_idx, (data, target) in enumerate(less_train_dataloader): # <-- now it is a distributed dataset
            model_on_worker = model_list[batch_idx%len(workers)]
            model_on_worker.train()
            model_on_worker.send(workers[batch_idx%len(workers)]) # <-- NEW: send the model to the right location

            data_on_worker = data.send(workers[batch_idx%len(workers)])
            target_on_worker = target.send(workers[batch_idx%len(workers)])

            data_on_worker, target_on_worker = data_on_worker.to(device), target_on_worker.to(device)

            opt_list[batch_idx%len(workers)].zero_grad()

            output = model_on_worker(data_on_worker)
            loss = F.nll_loss(output, target_on_worker)
            loss.backward()

            opt_list[batch_idx%len(workers)].step()
            model_on_worker.get() # <-- NEW: get the model back

            model_list[batch_idx%len(workers)] = model_on_worker #When len(dataloader) is longer than the len(worker) send and get must be modified
            #model_list here is full of the model which has trained on the workers, there are all different now.

        if epoch % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, n_train_items, n_train_items ,
                100. * epoch / args.epochs, loss.item()))


    ##Aggregation time
    new_model = []
    tmp_model = Net().to(device)
    with torch.no_grad():
        for p in model_list[0].parameters():
            new_model.append(0)
            
        for m in model_list:
            for par_idx, par in enumerate(m.parameters()):
                #average the model_list
                new_model[par_idx] = new_model[par_idx]+par.data
                # we get new model in list format and need to set_ to model
        
        for i in range(1):
            for n in new_model:
                n.add_(torch.normal(0,1.3*1.5,n.size(),device=device)*args.lr/args.batch_size)
        
        for worker in range(len(workers)):
            for par_idx in range(len(new_model)):
                list(model_list[worker].parameters())[par_idx].set_(new_model[par_idx]/len(workers))
        #init model with new_model
        


In [12]:
def test(args, model, device, test_loader, r):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader)*(args.test_batch_size)
    accuracy = 100. * correct / (len(test_loader)*args.test_batch_size)
    #Since the test loader here is a list, we can get the len by * it with batch.size
    
    
    writer.add_scalar('Accuracy', accuracy,r)
    writer.add_scalar('Loss', test_loss, r)
    print('\nTest set in round{}: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        r , test_loss, correct, len(test_loader)* (args.test_batch_size),
        accuracy))

In [ ]:
%%time
#optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment
logdir = '/root/notebooks/tensorflow/logs/10%DP/'
writer = SummaryWriter(logdir)

model_list = []
model_list = model_init(workers, Net().to(device))
opt_list = opt_init(model_list)
# not finish in train, finish latter
pars = [list(model.parameters()) for model in model_list]

for r in range(1, args.rounds + 1):
    train(args, device, train_loader, opt_list, workers)
    print("After training")
    test(args, model_list[0], device, test_loader, r)


if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 0 [6000/6000 (0%)]	Loss: 2.190672
After training

Test set in round1: Average loss: 2.1787, Accuracy: 5103/10020 (50.93%)

Train Epoch: 0 [6000/6000 (0%)]	Loss: 2.030240
After training

Test set in round2: Average loss: 2.0191, Accuracy: 7318/10020 (73.03%)

Train Epoch: 0 [6000/6000 (0%)]	Loss: 1.795062
After training

Test set in round3: Average loss: 1.7338, Accuracy: 7679/10020 (76.64%)

Train Epoch: 0 [6000/6000 (0%)]	Loss: 1.324450
After training

Test set in round4: Average loss: 1.2784, Accuracy: 8048/10020 (80.32%)

Train Epoch: 0 [6000/6000 (0%)]	Loss: 0.927186
After training

Test set in round5: Average loss: 0.8805, Accuracy: 8413/10020 (83.96%)

Train Epoch: 0 [6000/6000 (0%)]	Loss: 0.842654
After training

Test set in round6: Average loss: 0.7325, Accuracy: 7985/10020 (79.69%)

Train Epoch: 0 [6000/6000 (0%)]	Loss: 0.561933
After training

Test set in round7: Average loss: 0.5757, Accuracy: 8483/10020 (84.66%)

Train Epoch: 0 [6000/6000 (0%)]	Loss: 0.539166
A